# Implementing mixed precision 
Here we explore executing layer computations in float16 while keeping intermediary variables as float32. This decreases training time by 1.2 times for Resnet20 and Resnet44 for Cifar100 without impairing accuracy.

## Installs

In [ ]:
!pip install tensorflow_model_optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 238 kB 12.3 MB/s 


In [ ]:
from __future__ import print_function
import keras
import tempfile
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import CSVLogger  #, UpdatePruningStep
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_callbacks
import tensorflow_model_optimization as tfmot
import numpy as np
import time
import os
import pickle
from keras.datasets import cifar100
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision

## Model Setup

In [ ]:
# timing per epoch iteration
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [ ]:
# Learning rate scheduler 
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:
# define builder function for resnet layers
def resnet_layer_mixed_prec(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    print(conv.dtype_policy)
    print('x.dtype: %s' % x.dtype.name)
    # 'kernel' is dense1's variable
    print('dense1.kernel.dtype: %s' % conv.kernel.dtype.name)
    return x

In [ ]:
# actual resnet code
def resnet_mixed_prec(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer_mixed_prec(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer_mixed_prec(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer_mixed_prec(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer_mixed_prec(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    # x = AveragePooling2D(pool_size=8)(x)
    # y = Flatten()(x)
    # outputs = Dense(num_classes,
    #                 activation='softmax',
    #                 kernel_initializer='he_normal')(y)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal',
                    dtype='float32')(y)
    print('Outputs dtype: %s' % outputs.dtype.name)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# actual trainer function
def resnet_training_mixed_prec(x_train,y_train,x_test, y_test,layers,frequency,\
                    initial_sparsity,final_sparsity,gpu="T4",\
                    const=True, poly=False,file_name='cifar10_test1',\
                    num_classes=10,begin_step=0,end_step='default',block_size=(1,1)):

    # Default parameters
    batch_size = 128 
    epochs = 1
    data_augmentation = False #True
    n = layers

    # Computed depth from supplied model parameter n
    depth = n * 6 + 2

    # Input image dimensions.
    input_shape = x_train.shape[1:]

    # Subtracting pixel mean improves accuracy
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print('y_train shape:', y_train.shape)

#     ####Changes start#####
#     num_images = x_train.shape[0] #* (1 - validation_split)

#     if end_step == 'default':
#         end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

#     #hyperparameters: initial_sparsity=0.50, final_sparsity=0.80
#     if poly:
#         pruning_params = {
#               'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
#                                                                        final_sparsity=final_sparsity,
#                                                                        begin_step=begin_step,
#                                                                        end_step=end_step,
#                                                                       frequency=frequency),
#             'block_size': block_size
#         }
#     if const:
#             pruning_params = {
#               'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=final_sparsity,
#                                                                        begin_step=begin_step,
#                                                                        end_step=end_step,
#                                                                        frequency=frequency),
#                 'block_size': block_size
#         }

#     prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
    model = resnet_mixed_prec(input_shape=input_shape, depth=depth,num_classes=num_classes)
#    model = prune_low_magnitude(model, **pruning_params)    #_for_pruning

    model.compile(loss= 'categorical_crossentropy',     #''  tf.  keras.losses.SparseCategoricalCrossentropy(from_logits=True)
                  optimizer=Adam(lr=lr_schedule(0)),
                  metrics=['accuracy'])
    ####Changes end#####
    model.summary()

    # Prepare model model saving directory.
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)

    model_run=file_name+'_'+str(depth)+'_'+gpu
    model_path=model_run+'.h5'
    print("RRrrrrrR",model_path)
    
    filepath = os.path.join(save_dir, model_path)
    print("RRR",filepath)
    
    print(model_run)

    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(filepath=filepath,
                                 monitor='val_acc',
                                 verbose=1,
                                 save_best_only=True)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                   cooldown=0,
                                   patience=5,
                                   min_lr=0.5e-6)
    logdir = tempfile.mkdtemp()
    time_callback = TimeHistory()
    logname='/home/kal2257/saved_models/log_'+model_run+'.csv'
    csv_logger = CSVLogger(logname, append=True, separator=';')
    callbacks = [checkpoint, lr_reducer, lr_scheduler, csv_logger, time_callback]

    st = time.time()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
    #%tensorboard --logdir={logdir}
    training_time = time.time() - st

    print(f"overall training time is {training_time}")
    epoch_times = time_callback.times
    print(f"each epoch training time is {epoch_times}")

    # Score trained model.
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    print("tt",save_dir+'/'+model_run)
    
    #save standard model 
    model.save(save_dir+'/'+model_path)
    
    #saving data 
    with open(model_run+'.pickle', 'wb') as handle:
        pickle.dump([training_time,epoch_times,scores[0],scores[1]], handle)

     
    size=get_gzipped_model_size(save_dir+'/'+model_path) 
    print("Size of gzipped Keras model: %.2f bytes" % (size))
     
    with open(model_run+'.pickle', 'wb') as handle:
        pickle.dump([training_time,epoch_times,scores[0],scores[1],size], handle)

    return(model,model_path)

## Data: loading and preparation

In [ ]:
# Load the CIFAR10 data.
(X_train, Y_train), (X_test, Y_test) = cifar100.load_data()
# Normalize data.
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# Convert class vectors to binary class matrices.
Y_train = keras.utils.to_categorical(Y_train)
Y_test = keras.utils.to_categorical(Y_test)

169001437/169001437 [==============================] - 6s 0us/step


#Implementing Mixed Precision Training

In [ ]:
# Setting global policy to float 16 operations
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [ ]:
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


In [ ]:
#Demo-ing for resnet 20 and 44 baseline, no pruning.
layers=[3,7]
names=["twenty_mixed_prec","forty_four_mixed_prec"]

for x in range(len(layers)):
    name="resnet_"+names[x]
    print(name)
    model1 = resnet_training_mixed_prec(X_train,Y_train,X_test, Y_test,layers=layers[x],frequency=False,\
                    initial_sparsity = False, final_sparsity=False,gpu="T4",\
                    file_name=name,\
                    num_classes=100)